In [7]:
from openai import OpenAI
from openai import AzureOpenAI

from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery



In [2]:

load_dotenv(override=True) # take environment variables from .env.

endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_QUERY_KEY"]) 
index_name = os.environ["AZURE_SEARCH_INDEX"]


# Set the API key and endpoint
api_key = os.getenv('AZURE_OPENAI_API_KEY')
api_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')  # e.g., "https://<your-resource-name>.openai.azure.com/"
api_type = 'azure'
api_version = '2023-05-15'  # Use the appropriate API version

# Define the deployment name
deployment_name_chat = 'gpt-4o-global'
deployment_name_embeddings = 'text-embedding-ada-002'


In [3]:
import re 
def get_page_number(chunk_id : str): 
    page_re = r'_pages_(\d+)$'

    match = re.search(page_re, chunk_id)
    if match:
        page_number = match.group(1)
        return page_number

In [5]:
# Set query parameters for grounding the conversation on your search index
search_type="text"
use_semantic_reranker=True
sources_to_include=5

In [6]:
client = AzureOpenAI(
    azure_endpoint=api_endpoint,
    api_key=api_key,
    api_version=api_version,
)

In [8]:
search_client = SearchClient(
    endpoint, 
    index_name, 
    credential=credential)



In [79]:
 # This prompt provides instructions to the model
 GROUNDED_PROMPT="""
 You are a friendly assistant that answer questions from company's HR policy and information documents.
 Answer the query using only the sources provided below in a friendly and concise bulleted manner.
 Answer ONLY with the facts listed in the list of sources below.
 If there isn't enough information below, say you don't know.
 Do not generate answers that don't use the sources below.
 always provide the source of the information as a citation of source number in the list of sources.
 Sources:\n{sources}
 """

In [76]:

 # Query is the question being asked. It's sent to the search engine and the LLM.
user_query="Am I covered for eye tests?"


In [63]:
 # Retrieve the selected fields from the search index related to the question.
from azure.search.documents.models import (
    QueryType,
    QueryCaptionType,
    QueryAnswerType
)


vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=1, fields="vector", exhaustive=True)


results = search_client.search(  
    search_text=query,
    vector_queries=[vector_query],
    select=["parent_id", "chunk_id", "chunk", "title"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name='my-semantic-config',
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=sources_to_include
)

semantic_answers = results.get_answers()

In [64]:
list_of_sources = [f'\nSOURCE NO {index + 1}. {result["title"]}({get_page_number(result["chunk_id"])}): {result["chunk"]} )' for index, result in enumerate(results)]

In [65]:
print(list_of_sources)

['\nSOURCE NO 1. Northwind_Health_Plus_Benefits_Details.pdf(104): be covered by your Northwind Health Plus plan. \n\nWhen Prior Authorization is Required \n\nPrior authorization is required for certain services and treatments such as:  \n\n• Hospital admissions \n\n• Inpatient surgery \n\n• Outpatient surgery \n\n• Magnetic Resonance Imaging (MRI) \n\n• Computed Tomography (CT) \n\n• Radiation Therapy \n\n• Durable Medical Equipment \n\n• Physical, Occupational, and Speech Therapy  \n\n• Home Health Care \n\n• Infusion Therapy \n\n• Prosthetics and Orthotics \n\n• Specialty Drugs  \n\n\n\nIn certain cases, Northwind Health may require prior authorization even if the service is not \n\nlisted above. Your doctor or health care provider should contact Northwind Health to \n\ndetermine if prior authorization is required prior to providing care. \n\nExceptions to Prior Authorization \n\nThere are certain services and treatments that are exempt from prior authorization. These \n\ninclude:  \

In [66]:
joined_sources = "\n".join(list_of_sources)

In [73]:
print(joined_sources)


SOURCE NO 1. Northwind_Health_Plus_Benefits_Details.pdf(104): be covered by your Northwind Health Plus plan. 

When Prior Authorization is Required 

Prior authorization is required for certain services and treatments such as:  

• Hospital admissions 

• Inpatient surgery 

• Outpatient surgery 

• Magnetic Resonance Imaging (MRI) 

• Computed Tomography (CT) 

• Radiation Therapy 

• Durable Medical Equipment 

• Physical, Occupational, and Speech Therapy  

• Home Health Care 

• Infusion Therapy 

• Prosthetics and Orthotics 

• Specialty Drugs  



In certain cases, Northwind Health may require prior authorization even if the service is not 

listed above. Your doctor or health care provider should contact Northwind Health to 

determine if prior authorization is required prior to providing care. 

Exceptions to Prior Authorization 

There are certain services and treatments that are exempt from prior authorization. These 

include:  

• Routine office visits 

• Immunizations 



In [80]:
GROUNDED_PROMPT.format(sources=joined_sources)
print(GROUNDED_PROMPT.format(sources=joined_sources))


You are a friendly assistant that answer questions from company's HR policy and information documents.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
always provide the source of the information as a citation of source number in the list of sources.
Sources:

SOURCE NO 1. Northwind_Health_Plus_Benefits_Details.pdf(104): be covered by your Northwind Health Plus plan. 

When Prior Authorization is Required 

Prior authorization is required for certain services and treatments such as:  

• Hospital admissions 

• Inpatient surgery 

• Outpatient surgery 

• Magnetic Resonance Imaging (MRI) 

• Computed Tomography (CT) 

• Radiation Therapy 

• Durable Medical Equipment 

• Physical, Occupational, and Speech Therapy  

• Home Health Care 

• Infusion Therapy 

In [81]:
response = client.chat.completions.create(
    model=deployment_name_chat,
    messages=[
        {"role": "system", "content": GROUNDED_PROMPT.format(sources=joined_sources)},
        {"role": "user", "content": user_query}
    ]
)

In [82]:


print(response.choices[0].message.content)

Yes, eye tests are covered. Routine eye exams are exempt from prior authorization under the Northwind Health Plus plan (Source 1). Additionally, vision screenings are included as part of the preventive care services in the Northwind Standard plan (Source 2). 

For further details, you may want to check with Northwind Health customer service to ensure you have the latest information on your specific benefits.
